# 提取素数

In [ ]:
# 从文件中计算 top 素数 16_000
import sympy as sp
import sys
import re
sys.set_int_max_str_digits(2_000_000_000)

def calculate_expression(expression):
    # 将字符串表达式转换为Sympy表达式
    if "!" in expression:
        factorial_pattern = re.compile(r'(\d+)!')
    
        # 替换阶乘运算符为 sympy 的 factorial 函数
        expression = factorial_pattern.sub(r'sp.factorial(\1)', expression)
        # print(expression)
        return eval(expression)
    
    # print(expression)
    result = sp.sympify(expression)
    
    # 计算表达式的数值
    # result = expr.evalf()
    
    return result
    
def extrace_primes():
    """从文件中提取素数表格"""
    txt_path = "./primes_top_16000.txt"
    with open(txt_path,"rt")as f:
        rows = f.readlines()
        
    new_rows = []
    line = ""
    for row in rows:
        
        if row[:6].replace(" ","") == "":
            # 分行
            if line.endswith("\\"):
                line = line.rstrip("\\") + row.strip()
            else:
                line += row.rstrip()
        else:
            if line:
                data = line.split()
                new_rows.append((data[0],data[1],data[2]))
            line = row.rstrip()

    if line:
        data = line.split()
        new_rows.append((data[0],data[1],data[2]))
    return new_rows

def generate_top_primes():
    for rank, expression, digital in extrace_primes():
        try:
            value = calculate_expression(expression)
            yield expression, rank, digital, int(value), sp.N(value, 15)
        except Exception as e:
            print("calculate_expression error", rank, expression)
            print(e)
            continue
        
new_rows = extrace_primes()
new_rows[778]

# 编码数值

In [ ]:
def encode_number(big_int):
    byte_length = (big_int.bit_length() + 7) // 8
    big_int_byte = big_int.to_bytes(length=byte_length, byteorder="big")
    return big_int_byte
    
def encode_number_str(num_str):
    return encode_number(int(num_str))
    
def decode_number(num_byte):
    integer_value = int.from_bytes(num_byte, 'big')
    return integer_value
    
ss = ""
for i in range(2):
    ss += "1234567890"
tmp = encode_number_str(ss)
decode_number(tmp)

# 压缩数值

In [ ]:
import pandas as pd
from compress_utils import compress_test_data_v3, set_df_info, compress_data

result = []
def get_prime():
    for comment,rank,digital,value,estimate in generate_top_primes():
        report = {
            "formular": comment,
            "rank": rank,
            "digital": digital,
            "estimate": estimate,
        }
        print(rank, end=" ")
        ret = compress_data(encode_number(int(value)))
        report.update(ret)
        result.append(report)
        
    df = pd.DataFrame(result)
    
    df["gz-first-ratio"] = df["pure"] / df["gz-first-size"]
    df["gz-last-ratio"] = df["pure"] / df["gz-last-size"]
    df["bz-first-ratio"] = df["pure"] / df["bz-first-size"]
    df["bz-last-ratio"] = df["pure"] / df["bz-last-size"]
    df["xz-first-ratio"] = df["pure"] / df["xz-first-size"]
    df["xz-last-ratio"] = df["pure"] / df["xz-last-size"]
    return df
        

data_frame = get_prime()
data_frame = set_df_info(data_frame, "big integer", "prime")
data_frame.to_csv("tmp_bigInteger.csv")